In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from utils import load_behance_item_ids_with_features
import pandas as pd
import numpy as np
import random
%aimport utils

In [28]:
!ls /mnt/workspace/Behance/

Behance_appreciate_1M	  Behance_Item_to_Owners  test_pos.npy	  train.npy
Behance_Image_Features.b  test_neg.npy		  test_users.npy


In [4]:
DATASET_PATH = '/mnt/workspace/Behance/'

In [5]:
tmp = load_behance_item_ids_with_features(DATASET_PATH + 'Behance_Image_Features.b')

bytes: 2930692096
n_items: 178787


In [6]:
item_ids = tmp['ids']
item_id2index = tmp['id2index']

In [7]:
item_ids_set = set(item_ids)
assert len(item_ids_set) == len(item_ids)

In [8]:
df_likes = pd.read_csv(DATASET_PATH + 'Behance_appreciate_1M', sep=" ",
                            header=None, names=['user', 'item', 'timestamp'])

In [9]:
df_likes = df_likes[df_likes.item.isin(item_ids_set)]
df_likes.sort_values('timestamp', inplace=True)

In [10]:
len(df_likes)

999999

In [11]:
user_ids = df_likes.user.unique()

In [12]:
train_rows = []
test_pos_rows = []
test_neg_rows = []

In [13]:
user2mint = { u:None for u in user_ids }
user2maxt = { u:None for u in user_ids }
for u, _, t in df_likes.values:
    mint = user2mint[u]
    maxt = user2maxt[u]
    user2mint[u] = t if mint is None else min(mint, t)
    user2maxt[u] = t if maxt is None else max(maxt, t)

In [14]:
testable_user_ids = set()

In [15]:
user2likedItems = { u:set() for u in user_ids }
old_t = -999999
for u, i, t in df_likes.values:
    assert old_t <= t
    old_t = t
    liked_items = user2likedItems[u]
    liked_items.add(i)
    mint = user2mint[u]
    maxt = user2maxt[u]
    assert mint <= t <= maxt
    if mint < t and t == maxt:
        test_pos_rows.append((u,i))
        testable_user_ids.add(u)
    else:
        train_rows.append((u,i))

In [17]:
len(testable_user_ids)

47559

In [16]:
N_NEG_SAMPLES_PER_USER = 40

In [18]:
for u in testable_user_ids:
    sample_set = set()
    liked_items = user2likedItems[u]
    for _ in range(N_NEG_SAMPLES_PER_USER):
        while True:
            i = random.choice(item_ids)
            if i in sample_set: continue
            if i in liked_items: continue
            sample_set.add(i)
            test_neg_rows.append((u,i))
            break

In [25]:
train_array = np.array(train_rows)
test_pos_array = np.array(test_pos_rows)
test_neg_array = np.array(test_neg_rows)
test_users_array = np.array(list(testable_user_ids))

In [26]:
train_array.shape, test_pos_array.shape, test_neg_array.shape, test_users_array.shape

((952438, 2), (47561, 2), (1902360, 2), (47559,))

In [27]:
np.save(DATASET_PATH + 'train.npy', train_array)
np.save(DATASET_PATH + 'test_pos.npy', test_pos_array)
np.save(DATASET_PATH + 'test_neg.npy', test_neg_array)
np.save(DATASET_PATH + 'test_users.npy', test_users_array)